In [ ]:
# Install pyfuzzy-toolbox
!pip install pyfuzzy-toolbox -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import fuzzy_systems as fs
from fuzzy_systems import SugenoSystem

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# ============================================================================
# Create Sugeno First-Order (Linear) System
# ============================================================================
system = SugenoSystem(order=1)

# Add input variables
system.add_input('x1', (0, 10))
system.add_term('x1', 'low', 'trapezoidal', (0, 0, 4, 6))
system.add_term('x1', 'medium', 'triangular', (4, 6, 8))
system.add_term('x1', 'high', 'trapezoidal', (6, 8, 10, 10))

system.add_input('x2', (0, 10))
system.add_term('x2', 'low', 'trapezoidal', (0, 0, 4, 6))
system.add_term('x2', 'medium', 'triangular', (4, 6, 8))
system.add_term('x2', 'high', 'trapezoidal', (6, 8, 10, 10))

# Add output (order 1 = linear)
system.add_output('y')

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 10))

x_vals = np.linspace(0, 10, 200)
colors = {'low': 'b', 'medium': 'y', 'high': 'r'}

# Variable x1
for term_name, term in system.input_variables['x1'].terms.items():
    mu = term.membership(x_vals)
    axes[0].plot(x_vals, mu, colors[term_name] + '-', linewidth=2, label=term_name.capitalize())

axes[0].set_title('Input Variable: x₁', fontsize=14, fontweight='bold')
axes[0].set_xlabel('x₁')
axes[0].set_ylabel('Membership Degree')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([-0.05, 1.05])

# Variable x2
for term_name, term in system.input_variables['x2'].terms.items():
    mu = term.membership(x_vals)
    axes[1].plot(x_vals, mu, colors[term_name] + '-', linewidth=2, label=term_name.capitalize())

axes[1].set_title('Input Variable: x₂', fontsize=14, fontweight='bold')
axes[1].set_xlabel('x₂')
axes[1].set_ylabel('Membership Degree')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([-0.05, 1.05])

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# Define Sugeno First-Order Rules with Linear Outputs
# ============================================================================
# For order 1, coefficients are: [p1, p2, ..., c]
# y = p1*x1 + p2*x2 + c

system.add_rules([
    ['low', 'low', [0.2, 0.1, 1.0]],        # y = 0.2·x1 + 0.1·x2 + 1.0
    ['medium', 'medium', [0.4, 0.3, 3.0]],  # y = 0.4·x1 + 0.3·x2 + 3.0
    ['high', 'high', [0.6, 0.5, 5.0]]       # y = 0.6·x1 + 0.5·x2 + 5.0
])

In [ ]:
print("=" * 60)
print("TEST: x1=6.5, x2=7.0")
print("=" * 60)

x1_val, x2_val = 6.5, 7.0
result = system.evaluate({'x1': x1_val, 'x2': x2_val})
y = result['y']

print(f"\nInputs: x1={x1_val}, x2={x2_val}")
print(f"Output: y = {y:.3f}")

# Show membership degrees
memberships_x1 = system.input_variables['x1'].fuzzify(x1_val)
print(f"\nMembership degrees (x1={x1_val}):")
for term, mu in memberships_x1.items():
    print(f"  {term}: {mu:.3f}")

print("=" * 60)

In [ ]:
# Create grid
x1_vals = np.linspace(0, 10, 40)
x2_vals = np.linspace(0, 10, 40)
X1, X2 = np.meshgrid(x1_vals, x2_vals)

# Calculate outputs
Y = np.zeros_like(X1)
for i in range(X1.shape[0]):
    for j in range(X1.shape[1]):
        result = system.evaluate({'x1': X1[i, j], 'x2': X2[i, j]})
        Y[i, j] = result['y']

# Plot
fig = plt.figure(figsize=(14, 6))

ax1 = fig.add_subplot(121, projection='3d')
surf = ax1.plot_surface(X1, X2, Y, cmap='viridis', alpha=0.9)
ax1.set_xlabel('x₁')
ax1.set_ylabel('x₂')
ax1.set_zlabel('y')
ax1.set_title('3D Surface - Sugeno First-Order', fontweight='bold')
ax1.view_init(elev=25, azim=135)
fig.colorbar(surf, ax=ax1, shrink=0.5)

ax2 = fig.add_subplot(122)
contour = ax2.contourf(X1, X2, Y, levels=15, cmap='viridis')
ax2.set_xlabel('x₁')
ax2.set_ylabel('x₂')
ax2.set_title('Contour Map', fontweight='bold')
fig.colorbar(contour, ax=ax2)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Zero-Order System
system_o0 = SugenoSystem(order=0)
system_o0.add_input('x', (0, 10))
system_o0.add_term('x', 'low', 'trapezoidal', (0, 0, 4, 6))
system_o0.add_term('x', 'medium', 'triangular', (4, 6, 8))
system_o0.add_term('x', 'high', 'trapezoidal', (6, 8, 10, 10))
system_o0.add_output('y')
system_o0.add_rules([
    ('low', 2.0),
    ('medium', 6.0),
    ('high', 9.0)
])

# First-Order System
system_o1 = SugenoSystem(order=1)
system_o1.add_input('x', (0, 10))
system_o1.add_term('x', 'low', 'trapezoidal', (0, 0, 4, 6))
system_o1.add_term('x', 'medium', 'triangular', (4, 6, 8))
system_o1.add_term('x', 'high', 'trapezoidal', (6, 8, 10, 10))
system_o1.add_output('y')
system_o1.add_rules([
    ('low', [0.2, 1.0]),      # y = 1 + 0.2x
    ('medium', [0.4, 3.0]),   # y = 3 + 0.4x
    ('high', [0.6, 5.0])      # y = 5 + 0.6x
])

# Calculate curves
x_vals = np.linspace(0, 10, 100)
y_order0 = [system_o0.evaluate({'x': x})['y'] for x in x_vals]
y_order1 = [system_o1.evaluate({'x': x})['y'] for x in x_vals]

plt.figure(figsize=(12, 6))
plt.plot(x_vals, y_order0, 'b-', linewidth=3, label='Sugeno Order 0')
plt.plot(x_vals, y_order1, 'r-', linewidth=3, label='Sugeno Order 1')
plt.xlabel('Input (x)', fontsize=12)
plt.ylabel('Output (y)', fontsize=12)
plt.title('Comparison: Order 0 vs Order 1', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Order 0: Step-like response (piecewise constant)")
print("Order 1: Smooth, linear response (continuous)")